In [11]:
# ========== 超级内存节省版本 ==========
import pandas as pd
import numpy as np
import gc
import warnings
warnings.filterwarnings('ignore')

# 🔥 极端配置
TRAIN_SAMPLE_FRAC = 0.5  # 🔥 只用50%数据
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# 🔥 设置pandas内存优化
pd.set_option('mode.copy_on_write', True)

print(f"🔥 超级内存节省模式")
print(f"📊 采样比例: {TRAIN_SAMPLE_FRAC*100}%")

🔥 超级内存节省模式
📊 采样比例: 50.0%


In [12]:
# ========== 分块加载数据 ==========
print("🚀 分块加载数据策略")

def load_data_in_chunks():
    """分块加载并立即采样"""
    
    # 🔥 方法1: 先获取ranker_id列表
    print("📥 第一步：只读取ranker_id列...")
    
    # 只读取ranker_id列
    train_ids = pd.read_parquet(
        '/kaggle/input/aeroclub-recsys-2025/train.parquet', 
        columns=['ranker_id']
    )
    
    print(f"   原始ranker_id数量: {train_ids['ranker_id'].nunique():,}")
    
    # 🔥 立即采样ranker_id
    unique_rankers = train_ids['ranker_id'].unique()
    n_sample = int(len(unique_rankers) * TRAIN_SAMPLE_FRAC)
    sampled_rankers = np.random.RandomState(RANDOM_STATE).choice(
        unique_rankers, size=n_sample, replace=False
    )
    
    print(f"   采样ranker_id数量: {len(sampled_rankers):,}")
    
    # 🔥 删除不需要的数据
    del train_ids, unique_rankers
    gc.collect()
    
    # 🔥 第二步：只读取必要的列
    print("📥 第二步：读取必要列...")
    
    # 只读取最基础的列
    essential_cols = [
        'Id', 'ranker_id', 'selected', 'totalPrice', 'taxes',
        'legs0_duration', 'legs1_duration', 'isVip',
        'nationality', 'searchRoute',
        'legs0_segments0_marketingCarrier_code',
        'legs0_segments0_arrivalTo_airport_iata',
        'legs0_segments0_departureFrom_airport_iata'
    ]
    
    # 读取训练数据的必要列
    train = pd.read_parquet(
        '/kaggle/input/aeroclub-recsys-2025/train.parquet',
        columns=essential_cols
    )
    
    # 🔥 立即过滤到采样的ranker_id
    train = train[train['ranker_id'].isin(sampled_rankers)].copy()
    
    print(f"   采样后训练数据: {train.shape}")
    
    # 🔥 读取测试数据的对应列
    test_cols = [col for col in essential_cols if col != 'selected']
    test = pd.read_parquet(
        '/kaggle/input/aeroclub-recsys-2025/test.parquet',
        columns=test_cols
    )
    
    print(f"   测试数据: {test.shape}")
    
    # 🔥 立即优化数据类型
    for col in ['ranker_id', 'nationality', 'searchRoute', 
                'legs0_segments0_marketingCarrier_code',
                'legs0_segments0_arrivalTo_airport_iata', 
                'legs0_segments0_departureFrom_airport_iata']:
        if col in train.columns:
            train[col] = train[col].astype('category')
        if col in test.columns:
            test[col] = test[col].astype('category')
    
    # 转换数值列为更小的类型
    for col in ['totalPrice', 'taxes']:
        if col in train.columns:
            train[col] = train[col].astype(np.float32)
        if col in test.columns:
            test[col] = test[col].astype(np.float32)
    
    for col in ['selected', 'isVip']:
        if col in train.columns:
            train[col] = train[col].astype(np.int8)
        if col in test.columns:
            test[col] = test[col].astype(np.int8)
    
    return train, test

# 执行数据加载
train, test = load_data_in_chunks()

print(f"✅ 数据加载完成")
print(f"   训练数据: {train.shape}")
print(f"   测试数据: {test.shape}")
print(f"   内存使用显著减少")

# 立即清理
gc.collect()

🚀 分块加载数据策略
📥 第一步：只读取ranker_id列...
   原始ranker_id数量: 105,539
   采样ranker_id数量: 52,769
📥 第二步：读取必要列...
   采样后训练数据: (9123530, 13)
   测试数据: (6897776, 12)
✅ 数据加载完成
   训练数据: (9123530, 13)
   测试数据: (6897776, 12)
   内存使用显著减少


0

In [13]:
# ========== 终极简化特征工程 ==========
print("🚀 终极简化特征工程")

def ultra_simple_features(df):
    """终极简化 - 0秒完成"""
    
    print(f"   处理数据: {df.shape}")
    
    # 🔥 只要最基础的特征，不做任何复杂计算
    
    # 1. 价格对数
    df['log_price'] = np.log1p(df['totalPrice']).astype(np.float32)
    
    # 2. 税费比例
    df['tax_ratio'] = (df['taxes'] / (df['totalPrice'] + 1)).astype(np.float32)
    
    # 3. 组内价格排名 - 这是最重要的特征
    print("   计算价格排名...")
    df['price_rank'] = df.groupby('ranker_id')['totalPrice'].rank().astype(np.int16)
    
    # 4. 是否最便宜
    print("   标记最便宜选项...")
    price_min = df.groupby('ranker_id')['totalPrice'].transform('min')
    df['is_cheapest'] = (df['totalPrice'] == price_min).astype(np.int8)
    
    print(f"   ✅ 完成，新增4个特征")
    return df

# 🔥 快速应用
print("🔧 训练数据...")
train = ultra_simple_features(train)

print("🔧 测试数据...")
test = ultra_simple_features(test)

print(f"✅ 特征工程完成！")
print(f"   训练数据: {train.shape}")
print(f"   测试数据: {test.shape}")

# 清理
import gc
gc.collect()

🚀 终极简化特征工程
🔧 训练数据...
   处理数据: (9123530, 13)
   计算价格排名...
   标记最便宜选项...
   ✅ 完成，新增4个特征
🔧 测试数据...
   处理数据: (6897776, 12)
   计算价格排名...
   标记最便宜选项...
   ✅ 完成，新增4个特征
✅ 特征工程完成！
   训练数据: (9123530, 17)
   测试数据: (6897776, 16)


0

In [14]:
# ========== 快速模型训练 ==========
print("🚀 快速模型训练")

# 🔥 只使用最少的特征
feature_cols = [
    'totalPrice', 'taxes', 'legs0_duration', 'legs1_duration',
    'log_price', 'total_duration', 'is_one_way', 
    'price_rank', 'is_cheapest', 'isVip',
    'nationality', 'searchRoute', 'legs0_segments0_marketingCarrier_code'
]

# 确保特征存在
feature_cols = [col for col in feature_cols if col in train.columns and col in test.columns]

print(f"📊 使用特征: {len(feature_cols)} 个")

# 准备数据
X_train = train[feature_cols].copy()
y_train = train['selected'].copy()
groups_train = train['ranker_id'].copy()

X_test = test[feature_cols].copy()
groups_test = test['ranker_id'].copy()

# 🔥 删除原始数据释放内存
del train, test
gc.collect()

print(f"   X_train: {X_train.shape}")
print(f"   X_test: {X_test.shape}")

# 🔥 简单验证分割
from sklearn.model_selection import train_test_split

# 按照80/20分割，不用GroupShuffleSplit节省内存
train_size = int(0.8 * len(X_train))
X_tr = X_train[:train_size].copy()
X_val = X_train[train_size:].copy()
y_tr = y_train[:train_size].copy()
y_val = y_train[train_size:].copy()
groups_tr = groups_train[:train_size].copy()
groups_val = groups_train[train_size:].copy()

# 🔥 删除完整训练数据
del X_train, y_train, groups_train
gc.collect()

print(f"   分割完成 - Train: {len(X_tr)}, Val: {len(X_val)}")

🚀 快速模型训练
📊 使用特征: 11 个
   X_train: (9123530, 11)
   X_test: (6897776, 11)
   分割完成 - Train: 7298824, Val: 1824706


In [15]:
# ========== 修复数据类型并训练XGBoost ==========
print("🚀 修复数据类型并训练XGBoost")

# 🔥 首先修复所有数据类型问题
def fix_data_types(X_tr, X_val, X_test):
    """修复所有数据类型问题"""
    print("🔧 修复数据类型...")
    
    # 🔥 创建副本避免修改原数据
    X_tr = X_tr.copy()
    X_val = X_val.copy() 
    X_test = X_test.copy()
    
    # 处理duration列 - 如果是object，转换为0
    for col in ['legs0_duration', 'legs1_duration']:
        if col in X_tr.columns:
            if X_tr[col].dtype == 'object':
                print(f"   修复 {col} 列...")
                X_tr[col] = pd.to_numeric(X_tr[col], errors='coerce').fillna(0).astype(np.float32)
                X_val[col] = pd.to_numeric(X_val[col], errors='coerce').fillna(0).astype(np.float32)
                X_test[col] = pd.to_numeric(X_test[col], errors='coerce').fillna(0).astype(np.float32)
    
    # 🔥 强制转换所有非数值列
    for col in X_tr.columns:
        if X_tr[col].dtype in ['object', 'category'] or str(X_tr[col].dtype).startswith('string'):
            print(f"   转换类别列 {col}...")
            # 对所有object/category列进行Label Encoding
            all_vals = pd.concat([
                X_tr[col].astype(str), 
                X_val[col].astype(str), 
                X_test[col].astype(str)
            ]).unique()
            
            # 创建映射字典
            mapping = {val: i for i, val in enumerate(all_vals)}
            
            # 应用映射并转换为数值类型
            X_tr[col] = X_tr[col].astype(str).map(mapping).fillna(-1).astype(np.int32)
            X_val[col] = X_val[col].astype(str).map(mapping).fillna(-1).astype(np.int32)
            X_test[col] = X_test[col].astype(str).map(mapping).fillna(-1).astype(np.int32)
            
        elif X_tr[col].dtype == 'bool':
            # 布尔转换为int
            X_tr[col] = X_tr[col].astype(np.int8)
            X_val[col] = X_val[col].astype(np.int8)
            X_test[col] = X_test[col].astype(np.int8)
        
        # 🔥 确保所有数值列都是float32或int类型
        elif X_tr[col].dtype in ['float64']:
            X_tr[col] = X_tr[col].astype(np.float32)
            X_val[col] = X_val[col].astype(np.float32)
            X_test[col] = X_test[col].astype(np.float32)
    
    # 🔥 最后检查：确保没有object类型的列
    for col in X_tr.columns:
        if X_tr[col].dtype == 'object':
            print(f"   🚨 发现遗漏的object列: {col}, 强制转换为数值")
            X_tr[col] = pd.to_numeric(X_tr[col], errors='coerce').fillna(0).astype(np.float32)
            X_val[col] = pd.to_numeric(X_val[col], errors='coerce').fillna(0).astype(np.float32)
            X_test[col] = pd.to_numeric(X_test[col], errors='coerce').fillna(0).astype(np.float32)
    
    # 填充所有NaN
    X_tr = X_tr.fillna(0)
    X_val = X_val.fillna(0)
    X_test = X_test.fillna(0)
    
    print(f"   ✅ 数据类型修复完成")
    print(f"   X_tr形状: {X_tr.shape}")
    print(f"   数据类型分布: {X_tr.dtypes.value_counts().to_dict()}")
    
    # 🔥 最终验证：检查是否还有非数值类型
    non_numeric_cols = []
    for col in X_tr.columns:
        if not pd.api.types.is_numeric_dtype(X_tr[col]):
            non_numeric_cols.append(col)
    
    if non_numeric_cols:
        print(f"   ⚠️ 警告：仍有非数值列: {non_numeric_cols}")
        # 强制转换剩余的列
        for col in non_numeric_cols:
            X_tr[col] = pd.to_numeric(X_tr[col], errors='coerce').fillna(0).astype(np.float32)
            X_val[col] = pd.to_numeric(X_val[col], errors='coerce').fillna(0).astype(np.float32)
            X_test[col] = pd.to_numeric(X_test[col], errors='coerce').fillna(0).astype(np.float32)
    
    return X_tr, X_val, X_test

# 修复数据类型
X_tr, X_val, X_test = fix_data_types(X_tr, X_val, X_test)

# 🔥 创建简化的组信息
print("📊 创建组信息...")

# 正确计算组大小
def get_group_sizes(groups):
    """获取正确的组大小"""
    return groups.value_counts().sort_index().values

group_sizes_tr = get_group_sizes(groups_tr)
group_sizes_val = get_group_sizes(groups_val)

print(f"   训练组数: {len(group_sizes_tr)}")
print(f"   验证组数: {len(group_sizes_val)}")
print(f"   训练数据总行数: {len(X_tr)}")
print(f"   组大小总和: {group_sizes_tr.sum()}")

# 🔥 创建DMatrix
print("🚀 创建DMatrix...")
try:
    # 🔥 第一步：检查数据类型
    print(f"   X_tr数据类型: {X_tr.dtypes.value_counts().to_dict()}")
    print(f"   是否有object类型: {(X_tr.dtypes == 'object').any()}")
    
    dtrain = xgb.DMatrix(X_tr, label=y_tr, group=group_sizes_tr)
    dval = xgb.DMatrix(X_val, label=y_val, group=group_sizes_val)
    print("   ✅ DMatrix创建成功")
except Exception as e:
    print(f"   ❌ DMatrix创建失败: {e}")
    print("   🔄 尝试启用分类特征支持...")
    
    try:
        dtrain = xgb.DMatrix(X_tr, label=y_tr, group=group_sizes_tr, enable_categorical=True)
        dval = xgb.DMatrix(X_val, label=y_val, group=group_sizes_val, enable_categorical=True)
        print("   ✅ 启用分类特征的DMatrix创建成功")
    except Exception as e2:
        print(f"   ❌ 启用分类特征也失败: {e2}")
        print("   🔄 尝试纯数值方法...")
        
        try:
            # 🔥 只保留数值列
            numeric_cols = X_tr.select_dtypes(include=[np.number]).columns
            print(f"   使用 {len(numeric_cols)} 个数值列")
            
            X_tr_numeric = X_tr[numeric_cols].fillna(0).astype(np.float32)
            X_val_numeric = X_val[numeric_cols].fillna(0).astype(np.float32)
            
            dtrain = xgb.DMatrix(X_tr_numeric, label=y_tr, group=group_sizes_tr)
            dval = xgb.DMatrix(X_val_numeric, label=y_val, group=group_sizes_val)
            print("   ✅ 数值列DMatrix创建成功")
            
            # 🔥 更新X_test以保持一致
            X_test = X_test[numeric_cols].fillna(0).astype(np.float32)
            
        except Exception as e3:
            print(f"   ❌ 数值列方法也失败: {e3}")
            print("   🔄 最后尝试：简化到二分类...")
            
            try:
                # 🔥 最后方案：放弃ranking，只用二分类
                X_tr_simple = X_tr.select_dtypes(include=[np.number]).fillna(0).values.astype(np.float32)
                X_val_simple = X_val.select_dtypes(include=[np.number]).fillna(0).values.astype(np.float32)
                
                dtrain = xgb.DMatrix(X_tr_simple, label=y_tr.values)
                dval = xgb.DMatrix(X_val_simple, label=y_val.values)
                print("   ✅ 简化二分类DMatrix创建成功")
                
                # 🔥 更新X_test
                X_test = pd.DataFrame(X_test.select_dtypes(include=[np.number]).fillna(0).values.astype(np.float32))
                
            except Exception as e4:
                print(f"   💥 所有DMatrix创建方法都失败了: {e4}")
                raise RuntimeError("无法创建XGBoost DMatrix，请检查数据")

# 🔥 超级简化的XGBoost参数
params = {
    'objective': 'binary:logistic',  # 🔥 改为二分类，更简单
    'eval_metric': 'logloss',
    'max_depth': 4,  # 🔥 很浅的树
    'learning_rate': 0.3,  # 🔥 更高的学习率
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'seed': RANDOM_STATE,
    'verbosity': 0,
    'n_jobs': 2  # 🔥 限制并行
}

print("🚀 开始训练...")
try:
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=50,  # 🔥 只训练50轮
        evals=[(dtrain, 'train'), (dval, 'val')],
        early_stopping_rounds=10,
        verbose_eval=10
    )
    print("✅ 训练成功完成")
except Exception as e:
    print(f"❌ 训练失败: {e}")
    # 如果还是失败，用最简单的参数
    print("🔄 尝试最简单的训练...")
    model = xgb.train(
        {'objective': 'binary:logistic', 'verbosity': 0},
        dtrain,
        num_boost_round=20
    )
    print("✅ 简化训练完成")

# 🔥 立即清理训练数据释放内存
del dtrain, X_tr, y_tr, groups_tr
if 'dval' in locals():
    del dval
gc.collect()

print("🎉 XGBoost训练步骤完成")

🚀 修复数据类型并训练XGBoost
🔧 修复数据类型...
   修复 legs0_duration 列...
   修复 legs1_duration 列...
   转换类别列 nationality...
   转换类别列 searchRoute...
   转换类别列 legs0_segments0_marketingCarrier_code...
   ✅ 数据类型修复完成
   X_tr形状: (7298824, 11)
   数据类型分布: {dtype('float32'): 5, dtype('int32'): 3, dtype('int8'): 2, dtype('int16'): 1}
📊 创建组信息...
   训练组数: 52769
   验证组数: 52769
   训练数据总行数: 7298824
   组大小总和: 7298824
🚀 创建DMatrix...
   X_tr数据类型: {dtype('float32'): 5, dtype('int32'): 3, dtype('int8'): 2, dtype('int16'): 1}
   是否有object类型: False
   ✅ DMatrix创建成功
🚀 开始训练...
[0]	train-logloss:0.10723	val-logloss:0.10954
[10]	train-logloss:0.02810	val-logloss:0.03199
[20]	train-logloss:0.02545	val-logloss:0.02951
[30]	train-logloss:0.02517	val-logloss:0.02922
[40]	train-logloss:0.02500	val-logloss:0.02904
[49]	train-logloss:0.02489	val-logloss:0.02890
✅ 训练成功完成
🎉 XGBoost训练步骤完成


In [16]:
# ========== 修复预测并创建提交文件 ==========
print("🚀 修复预测并创建提交文件")

# 🔥 重新读取test的Id列（因为可能丢失了）
print("📥 确保有Id列...")
try:
    test_ids = pd.read_parquet('/kaggle/input/aeroclub-recsys-2025/test.parquet', columns=['Id'])
    print(f"   ✅ 成功读取 {len(test_ids)} 个Id")
except Exception as e:
    print(f"   ⚠️ 无法读取Id列，使用索引: {e}")
    test_ids = pd.DataFrame({'Id': range(len(X_test))})

# 🔥 生成预测 - 修复group结构问题
print("🔮 生成预测...")
try:
    # 🔥 关键修复：不使用group信息创建测试DMatrix
    # 因为我们使用的是binary:logistic，不需要group信息
    print("   创建测试DMatrix（不使用group信息）...")
    dtest = xgb.DMatrix(X_test)  # 🔥 不传入group参数
    
    print("   开始预测...")
    test_preds = model.predict(dtest)
    print(f"   ✅ 生成了 {len(test_preds)} 个预测")
    
except Exception as e:
    print(f"   ❌ 标准预测失败: {e}")
    print("   🔄 尝试更简单的预测方法...")
    
    try:
        # 🔥 如果还是失败，直接使用numpy数组
        test_preds = model.predict(xgb.DMatrix(X_test.values))
        print(f"   ✅ 简化预测成功，生成了 {len(test_preds)} 个预测")
    except Exception as e2:
        print(f"   ❌ 简化预测也失败: {e2}")
        print("   ⚠️ 使用随机预测作为后备方案")
        test_preds = np.random.random(len(X_test))

# 🔥 创建提交DataFrame
print("📊 创建提交文件...")
submission = pd.DataFrame({
    'Id': test_ids['Id'],
    'ranker_id': groups_test,
    'pred_score': test_preds
})

print(f"   提交数据形状: {submission.shape}")
print(f"   预测值范围: {test_preds.min():.4f} 到 {test_preds.max():.4f}")

# 🔥 计算组内排名
print("🏆 计算组内排名...")
submission['selected'] = submission.groupby('ranker_id')['pred_score'].rank(
    ascending=False, method='first'
).astype(int)

print(f"   排名计算完成，范围: 1 到 {submission['selected'].max()}")

# 🔥 保存最终提交文件
output_file = 'ultra_fast_submission.csv'
# 🔥 修复：包含ranker_id列
final_submission = submission[['Id', 'ranker_id', 'selected']]
final_submission.to_csv(output_file, index=False)

print(f"🎉 任务完成！")
print(f"   📁 文件名: {output_file}")
print(f"   📊 预测数量: {len(final_submission):,}")
print(f"   🏆 排名范围: 1 到 {submission['selected'].max()}")

# 显示一些统计信息
print(f"\n📈 提交文件统计:")
print(f"   总行数: {len(final_submission)}")
print(f"   唯一Id数: {final_submission['Id'].nunique()}")
print(f"   唯一ranker_id数: {final_submission['ranker_id'].nunique()}")
print(f"   排名分布（前10）:")
rank_counts = final_submission['selected'].value_counts().head(10)
for rank in sorted(rank_counts.index):
    count = rank_counts[rank]
    print(f"     排名{rank}: {count:,} 个")

# 🔥 验证提交文件格式
print(f"\n🔍 验证提交文件:")
print(f"   必需列: {list(final_submission.columns)}")
print(f"   是否有重复Id: {final_submission['Id'].duplicated().any()}")
print(f"   selected列数据类型: {final_submission['selected'].dtype}")
print(f"   🔥 ranker_id列已包含: {'ranker_id' in final_submission.columns}")

# 🔥 显示提交文件示例
print(f"\n📋 提交文件示例（前5行）:")
print(final_submission.head().to_string(index=False))

# 🔥 最终清理
if 'model' in locals():
    del model
if 'dtest' in locals():
    del dtest
del X_test, submission, final_submission
gc.collect()

print(f"\n🚀 所有步骤完成！文件已保存到: {output_file}")
print("📝 可以直接提交这个CSV文件到Kaggle竞赛")

🚀 修复预测并创建提交文件
📥 确保有Id列...
   ✅ 成功读取 6897776 个Id
🔮 生成预测...
   创建测试DMatrix（不使用group信息）...
   开始预测...
   ✅ 生成了 6897776 个预测
📊 创建提交文件...
   提交数据形状: (6897776, 3)
   预测值范围: 0.0000 到 0.9654
🏆 计算组内排名...
   排名计算完成，范围: 1 到 7022
🎉 任务完成！
   📁 文件名: ultra_fast_submission.csv
   📊 预测数量: 6,897,776
   🏆 排名范围: 1 到 7022

📈 提交文件统计:
   总行数: 6897776
   唯一Id数: 6897776
   唯一ranker_id数: 45231
   排名分布（前10）:
     排名1: 45,231 个
     排名2: 44,347 个
     排名3: 43,120 个
     排名4: 41,915 个
     排名5: 41,102 个
     排名6: 40,615 个
     排名7: 39,389 个
     排名8: 38,946 个
     排名9: 38,282 个
     排名10: 37,258 个

🔍 验证提交文件:
   必需列: ['Id', 'ranker_id', 'selected']
   是否有重复Id: False
   selected列数据类型: int64
   🔥 ranker_id列已包含: True

📋 提交文件示例（前5行）:
      Id                        ranker_id  selected
18144679 c9373e5f772e43d593dd6ad2fa90f67a         1
18144680 c9373e5f772e43d593dd6ad2fa90f67a        52
18144681 c9373e5f772e43d593dd6ad2fa90f67a       150
18144682 c9373e5f772e43d593dd6ad2fa90f67a       154
18144683 c9373e5f772e43d593dd6a

In [17]:
# ========== 可选: 进一步分析 ==========
print("🔍 可选: 进一步分析")

# 预测分数分布
print("📊 预测分数统计:")
print(f"   最小值: {test_preds.min():.4f}")
print(f"   最大值: {test_preds.max():.4f}")
print(f"   均值: {test_preds.mean():.4f}")
print(f"   标准差: {test_preds.std():.4f}")

# 检查组大小分布
print(f"\n📏 测试集组大小分布:")
test_group_sizes = submission.groupby('ranker_id').size()
print(f"   平均组大小: {test_group_sizes.mean():.1f}")
print(f"   组大小范围: {test_group_sizes.min()}-{test_group_sizes.max()}")
print(f"   组大小众数: {test_group_sizes.mode().iloc[0]}")

size_dist = test_group_sizes.value_counts().sort_index()
print(f"   组大小分布（前10个）:")
for size in size_dist.head(10).index:
    print(f"     大小{size}: {size_dist[size]:,}组")

# 内存使用情况
print(f"\n💾 内存清理...")
del train, test, train_features, test_features, X_train, X_test
del X_tr, X_val, X_tr_encoded, X_val_encoded, X_test_encoded
del dtrain, dval, dtest
gc.collect()
print("✅ 内存清理完成")

print("\n🎉 所有分析完成！")

🔍 可选: 进一步分析
📊 预测分数统计:
   最小值: 0.0000
   最大值: 0.9654
   均值: 0.0073
   标准差: 0.0307

📏 测试集组大小分布:


NameError: name 'submission' is not defined